# First Assigment  FHPC course 

### Due date : 23.59.04.11.2019

***version 1.1 ( released at 13.00 28.11.2019)***

#### Submission policy:  by email to cozzini@iom.cnr.it as zip file named as Surname.FirstName.zip

Size of the file: less than 2MB
Structure of the files once decompressed:

	report.pdf
	codes/sumNumbers_mpi.[f90,c,cc]
	codes/readme

Report.pdf  will contain a detailed report on all the sections  in this documents. 
	
I strongly advise you to copy/edit  the structure of this  document  ( written in markdown format) with your preferred editor and keep the same structure of the document on your final report. Please make use of plots, tables to facilitate reading/understanding by us. 

Directory codes should includes all the programs we requested to write. The programs should be complemented by a readme file that explain which software stack you used to compile and run all the programs you provided  and on which machine. 


##Section 0: (warm-up)

- Compute Theoretical Peak performance for your laptop/desktop
  - Identify the CPU                       

  - Identify the frequency       

  - Identify how many cores   

  ​            Suggestion for Linux User:  `cat /proc/cpuinfo`  gives you all the information you neeed.

  - Identify the number of floating point for cycle  
    
      This is is tricky: try to find out some info about the global performance looking here:
      - https://en.wikipedia.org/wiki/FLOPS
      - https://stackoverflow.com/questions/23588380/how-to-compute-the-single-precision-data-and-double-precision-data-peak-performa?rq=1
  - Write here the  final result  obtained: 
  

  |        | Your model | CPU  | Frequency | number  of core | Peak performance |
  | ------ | ---------- | ---- | --------- | --------------- | ---------------- |
  | Laptop |            |      |           |                 |                  |


 

- Compute sustained Peak performance for your  cell-phone
  
  - Identify an app to run HPL, for instance:  https://play.google.com/store/apps/details?id=com.sqi.linpackbenchmark
  
  - Run&Tune it to get the best number you can 
  
    Suggestion: try to increase the size of the matrix
  
  - Extra Bonus:  Identify the CPU and calculate the Peak performance of the CPU  
  
  
  |            | Your model | sustained performance | size of matrix | Peak performance | memory |
  | ---------- | ---------- | --------------------- | -------------- | ---------------- | ------ |
  | SmartPhone |            |                       |                |                  |        |



- Find out in which year  your cell phone/laptop  could have been in top1% of Top500 

  - Fill in the following table

    

  |            | Your model | performance | Top500 year | number 1 HPC system | number of processors (TOP500) |
  | ---------- | ---------- | ----------- | ----------- | ------------------- | ----------------------------- |
  | SmartPhone |            |             |             |                     |                               |
  | Laptop     |            |             |             |                     |                               |


​    
## Section  1: theoretical model

- devise a performance model for a simple parallel algorithm: sum of N numbers

  - Serial Algorithm : n-1 operations 

    $T_{serial}= N*T_{comp}$ 
    $T_{comp}$= *time to compute a floating point operation*

  - Parallel Algorithm : master-slave	

    - read N and distribute N to P-1  slaves ===>  $T_{read} + (P-1)\times T_{comm}$ 
      $T_{comm}$ = *time  each processor takes to communicate one message, i.e. latency..*
      $T_{read}$   = *time  master takes to read* 

    - N/P sum over each processors (including master)  ===> $T_{comp}/N$

    - Slaves send partial sum  ===>   $(P-1)T_{comm}$

    - Master performs  one final sum ===>  $(P-1) \times $T_{comp}$

      the final model:    $T_p=   T_{comp}\times (P -1 + n/P)  + T_{read} + 2(P-1)\times T_{comm}  $

- compute scalability curves for such algorithm and make some plots

  - assumptions:

    - $T_{comp} =2 \times 10^{-9}$
    - $T_{read}= 1 \times 10^{-4}$
    - $T_{comm}= 1 \times 10^{-6}$

    Play with some value of N and plot against P  (with P ranging from 1 to 1000)

- Comment on them:

  - For which values of N do you see the algorithm scaling ? 

  - For which values of P does the algorithm produce the best results ?

  - Can you try to modify the algorithm sketched above to increase its scalability ? 

    (hints: try to think a  better communication  algorithm) 

  
  ![Sucks](foo.png)
  
  

## Section 2 : play with MPI program

### 2.1:  compute strong scalability of a mpi_pi.c program

In this section we will briefly describe the results of the analysis of the different execution time yielded by the code provided in class to compute PI using Monte-Carlo integration. The result refer both to the execution of the serial algorithm and to the parallel one.

#### Time measurement considerations

Firstly, it needs to be addressed the problem regarding the measure of time of an execution of a code. To measure the execution time of the serial algorithm, the idea is to use the command /usr/bin/time, as suggested by the assignment. This standard shell command yields three different measures which names are real, usr and sys. (#TODO Maybe a brief explanation of the three times). For a serial application, as in our case, we'd expect usr+sys to be approximatively equal to real, but, since we don't want to take into consideration the system time as per request of the assignment, we'll just use the usr time, i.e. the time employed by the cpu to run our program. 




  |              |  n=100000  |  n=1000000 |  n=10000000 |  n=100000000 |  n=1000000000 |  
  | ----------   | ---------- | ---------  |  ---------- |  ----------  |  ----------   | 
  | user time(s) |    0.003   |    0.021   |    0.197    |    1.966     |   19.665      |
  
_User time captured for the execution of the serial algorithm. Keeping into account some truncation (#TODO: maybe  avoid them) we can see that the user time scales linearly with n_
 

To identify the overhead of MPI routines, we can just consider the difference in between the usr time returned by the serial program, and the one returned by the parallel program using just one processor; since the computational part is identical in this case, this will yield us a rough estimate of the time involved in the communication, initialization and finalization routines.



  |                    |  n=100000  |  n=1000000 |  n=10000000 |  n=100000000 |  n=1000000000 |  
  | ----------         | ---------- | ---------  |  ---------- |  ----------  |  ----------   | 
  | user serial(s)     |    0.003   |    0.021   |    0.197    |    1.966     |   19.665      |
  | user paral(s), P=1 |    1.147   |    1.172   |    1.342    |    3.124     |   21.011      |
  | Difference         |    1.144   |    1.151   |    1.145    |    1.158     |    1.346      |
_User time captured both for the serial algorithm and for the parallel algorithm with one processor only. The difference between them remains pretty much constant with respect to n._



This approach cannot be valid also for our parallel code when we run it with multiple processor though, since the user time takes into consideration the sum of the times for every CPU. The only way in which we can actually have an extimation of the execution time of our parallel program (from now on denoted as T(P)) is to use the walltime provided by the MPI_Wtime calls inside the code. Of all the printed walltimes, our choice is to take the maximum one, since this is the measure of time of the processor that runs the longest. It is worth noting that this processor is not necessarily going to be the master, because of the way the communication are defined in the code. Since the choice of the programmer was to use a SSend routine in the code, we know from the theory that the last processor to send the information to the master will also be the slowest. This is due to the fact that this processor needs to wait for an answer from the master, confirming the actual reception of the message. Since the time of a single communication is roughly 10^3 slower than the time of a single computation, the master will finish doing the last sum while its confirmation message to the last sending processor is still being delivered. More on this will be


###Strong scaling

In order to verify the strong scaling of our code, we need to plot the different runtime values of our code against the number of processors used. We will acquire the data in the way that we've described in the previous paragraph, considering the maximum walltime out of all the processors for the previously mentioned reasons. We will also plot these data for 4 different values of N, increasing N by a factor of 10 each time; in this way it will be clear which is the size of N for which we can say that our code scales strongly.

////#TODO: insert graphs of plots of the actual runtime for different values of n




### 2.2: identify a model for the parallel overhead


////#TODO As defined in the slides of the course (at the moment I have no better way to say this), the parallel overhead is the syncronization time in between all the processors involved in a MPI program. In the previous paragraph we have seen a method to estimate it in the case of just one processor performing the parallel operation, using the difference in between the times of execution of the serial program vs the parallel algorithm with one processor. We can also try to understand what happens in case we've got more processor involved in the execution of the code by considering the walltime of every processor yielded by the code itself. We can then see that the master walltime is in fact the time it takes to do the actual PI computation, but not necessarily the time it takes for the program to finish! This is due to the fact that there are still communication ongoing in between the master and one or more of the slave processors. This communication is also the only one present in the program, since the input operation is done in parallel, unlike in our partial sum code. In conclusion, we just need need to take the difference in between the maximum of all the slaves processors walltime and the master walltime to get an extimation of our parallel overhead.

Theoretically speaking we should expect the following behaviour, considering only the communication part:

    1. Every node starts to send their partial calculation to the master once they finished.
    2. The master receives the result of the first node and it sums it to its local sum.
    3. This operation is repeated P-1 times. In the last loop, when the master receives the communication from the last processor, it sends back a confirmation of this. While the communication is sent to the last slave processor, the master performs the last sum, exits the loop and performs two multiplications.
    4. The last processor to send the information to the master, gets the confirmation and the program ends.
    
With this behaviour model, we should then expect the parallel overhead to be constant with respect to the variation of the processors, in particular it should have the value:

    Parallel_overhead = T_comm - (3*T_comp) circa = 10^-6 - 6*10^-9
    
Where the T_comm is the confirmation the slowest processor is waiting from the master, and 3*T_comp is the time the master takes to perform the last computation.


////#TODO: collect the data and plot them and hope for the best

### 2.3: weak scaling 

- Now let us do some running that constitutes a weak scaling test.

This means increasing the problem size simultaneously with the number of nodes being used. In the present case, increasing the number of iterations, Niter.
- Record the run time for each number of nodes and make a plot of the run time versus number of computing nodes.

- Weak scaling would imply that the runtime remains constant as the problem size and the number of compute nodes increase in proportion. Modify your scripts to rapidly collect numbers for the weak scalability tests for differrent number of moves.

- Plot on the same graph the efficiency (T(1)/T(p)) of weak scalability for different number of moves and comment the results obtained.

  

## Section 3: implement a parallel program using MPI 

I chose to write the program in C, since it's the language we've been using the most during these first hpc lectures. The code implements an algorithm that follows the model discussed during the lectures. In particular, it performs the following steps:

* Defines a master node that reads from command-line argument the number N of which we want to calculate the partial sum.

* Sends to P-1 slaves processor the number N/P. In particular, I've decided to communicate to every slave processor the number N, and make them perform two more calculation: n/p and n\%p. I've done this to solve the problem of redestribution of the remainder of N/P. In fact, calculating the remainder inside the master node and then splitting it to every slave node would have required another communication, thus increasing the actual performance of another P*T_comm, which is 10^3 times larger than P*T_comp. After calculating n\%p, every slave processor allocates a memory space one unit greater than the others, if its rank is smaller than the remainder obtained.

* Computes all the partial sum computation in every processor, master included

* Sends the partial results to the master which receives them and sums together the last P numbers

More technical information regarding the usage of the code will be given in the readme file provided.



## Section 4  (run and compare)

In this final section you are ruquested to run and measure strong scalability of  the program written in section 3.

  - Run the implemented program for different large enough sizes of the array 

  - Plot as in section 3 scalability of the program  

  - compare  performance results obtained against the performance model elaborated in section 2

      - comment on the assumption made in section 1 about times: are they correct ? do you observe something different ? 

        




3